In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#model.config.output_attentions = True  # 返回注意力权重
model.config.output_hidden_states = True  # 返回隐藏状态（可选）


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [2]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [16]:
prompt = "列出2020年1月份全球财经大事件"
messages = [
    {"role": "system", "content": "你是一名财经内容助手"},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text,text], return_tensors="pt").to(model.device)



In [17]:
generated_ids = model.generate(**model_inputs,max_new_tokens=512)

In [18]:

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

system
你是一名财经内容助手
user
列出2020年1月份全球财经大事件
assistant
以下是2020年1月份全球财经领域的一些重要事件：

1. 欧元兑美元汇率大幅波动：欧元兑美元在1月3日和5日分别触及了1.4987和1.4968，创下历史最高点。随后在1月11日再次回落至1.4950。

2. 美国通胀率上升：美国劳工部数据显示，截至1月1日当周，消费者价格指数（CPI）同比上涨了0.5%，创历史新高。

3. 韩国央行宣布降息：韩国央行决定将基准利率下调到-1%的水平，并表示将在未来几个月内继续采取宽松政策以应对通缩压力。

4. 英国央行加息：英国央行宣布于1月12日加息25个基点，达到0.1%的年内最大幅度。

5. 日本央行降息：日本央行宣布将于1月20日结束资产购买计划，停止每月购买超过1万亿日元的债券。

6. 中国股市出现反弹：A股市场在1月1日至2日之间呈现反弹态势，上证综指、深证成指以及创业板指等主要指数均实现不同程度的上涨。

7. 挪威央行宣布降息：挪威央行宣布自1月1日起将基准利率降至-0.5%，这是其历史上首次实施负利率措施。

8. 德国国债收益率上升：德国政府发行的国债收益率持续攀升，一度达到每欧元2.7%的高位。

这些事件反映了全球经济和金融市场的重要动态，同时也对各国货币政策、经济前景等方面产生了影响。


In [28]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [34]:
len(generated_ids['sequences'][0])

248

In [47]:
generated_ids['past_key_values'][1][1].shape

torch.Size([1, 2, 247, 64])